# SD212: Graph mining

## Lab 7: Graph neural networks

In this lab, you will learn to classify nodes using graph neural networks.

We use [DGL](https://www.dgl.ai) (deep graph library), which relies on pytorch.

In [138]:
# pip install dgl

## Import

In [2]:
import numpy as np
from scipy import sparse

In [3]:
from sknetwork.data import load_netset
from sknetwork.classification import DiffusionClassifier, get_accuracy_score
from sknetwork.embedding import Spectral
from sknetwork.utils import directed2undirected

In [4]:
import dgl
from dgl.nn import SAGEConv
from dgl import function as fn

In [5]:
import torch
from torch import nn
import torch.nn.functional as F

In [6]:
# ignore warnings from DGL
import warnings
warnings.filterwarnings('ignore')

## Load data

We will work on the following datasets (see the [NetSet](https://netset.telecom-paris.fr/) collection for details):
* Cora (directed graph + bipartite graph)
* WikiVitals (directed graph + bipartite graph)

Both datasets are graphs with node features (given by the bipartite graph) and ground-truth labels.

In [144]:
cora = load_netset('cora')
wikivitals = load_netset('wikivitals')

Parsing files...
Done.
Parsing files...
Done.


In [145]:
dataset = cora

In [146]:
adjacency = dataset.adjacency
biadjacency = dataset.biadjacency
labels = dataset.labels

In [147]:
# we use undirected graphs
adjacency = directed2undirected(adjacency)

In [148]:
# for Wikivitals, use spectral embedding of the bipartite graph as features

if dataset.meta.name.startswith('Wikivitals'):
    spectral = Spectral(50)
    features = spectral.fit_transform(biadjacency)
else:
    features = biadjacency.toarray()

In [149]:
def split_train_test_val(n_samples, test_ratio=0.1, val_ratio=0.1, seed=None):
    """Split the samples into train / test / validation sets.
    
    Returns
    -------
    train: np.ndarray
        Boolean mask
    test: np.ndarray
        Boolean mask
    validation: np.ndarray
        Boolean mask
    """
    if seed:
        np.random.seed(seed)

    # test
    index = np.random.choice(n_samples, int(np.ceil(n_samples * test_ratio)), replace=False)
    test = np.zeros(n_samples, dtype=bool)
    test[index] = 1
    
    # validation
    index = np.random.choice(np.argwhere(~test).ravel(), int(np.ceil(n_samples * val_ratio)), replace=False)
    val = np.zeros(n_samples, dtype=bool)
    val[index] = 1
    
    # train
    train = np.ones(n_samples, dtype=bool)
    train[test] = 0
    train[val] = 0
    return train, test, val

In [150]:
train, test, val = split_train_test_val(len(labels))

## Graph and tensors

In DGL, the graph is represented as an object, the features and labels as tensors.

In [151]:
# graph as an object
graph = dgl.from_scipy(adjacency)

In [152]:
type(graph)

dgl.heterograph.DGLGraph

In [153]:
# features and labels as tensors
features = torch.Tensor(features)
labels = torch.Tensor(labels).long()

In [154]:
# masks as tensors
train = torch.Tensor(train).bool()
test = torch.Tensor(test).bool()
val = torch.Tensor(val).bool()

## Graph neural network

We start with a simple graph neural network without hidden layer. The output layer is of type [GraphSAGE](https://docs.dgl.ai/generated/dgl.nn.pytorch.conv.SAGEConv.html).

In [155]:
class GNN(nn.Module):
    def __init__(self, dim_input, dim_output):
        super(GNN, self).__init__()
        self.conv = SAGEConv(dim_input, dim_output, 'mean')
        
    def forward(self, graph, features):
        output = self.conv(graph, features)
        return output

## To do

* Train the model on Cora and get accuracy.
* Compare with the same model trained on an empty graph.
* Add a hidden layer with ReLu activation function (e.g., dimension = 20) and retrain the model. 
* Compare with a classifier based on heat diffusion.

In [156]:
def init_model(model, features, labels):
    '''Init the GNN with appropriate dimensions.'''
    dim_input = features.shape[1]
    dim_output = len(labels.unique())
    return model(dim_input, dim_output)   

In [157]:
def eval_model(gnn, graph, features, labels, test=test):
    '''Evaluate the model in terms of accuracy.'''
    gnn.eval()
    with torch.no_grad():
        output = gnn(graph, features)
        labels_pred = torch.max(output, dim=1)[1]
        score = np.mean(np.array(labels[test]) == np.array(labels_pred[test]))
    return score

In [158]:
def train_model(gnn, graph, features, labels, train=train, val=val, n_epochs=100, lr=0.01, verbose=True):
    '''Train the GNN.'''
    optimizer = torch.optim.Adam(gnn.parameters(), lr=lr)
    
    gnn.train()
    scores = []
    
    for t in range(n_epochs):   
        
        # forward
        output = gnn(graph, features)
        logp = nn.functional.log_softmax(output, 1)
        loss = nn.functional.nll_loss(logp[train], labels[train])

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # evaluation
        score = eval_model(gnn, graph, features, labels, val)
        scores.append(score)
        
        if verbose and t % 10 == 0:
            print("Epoch {:02d} | Loss {:.3f} | Accuracy {:.3f}".format(t, loss.item(), score))

In [159]:
gnn = init_model(GNN, features, labels)

In [160]:
train_model(gnn, graph, features, labels)

Epoch 00 | Loss 1.986 | Accuracy 0.325
Epoch 10 | Loss 0.595 | Accuracy 0.797
Epoch 20 | Loss 0.247 | Accuracy 0.823
Epoch 30 | Loss 0.134 | Accuracy 0.838
Epoch 40 | Loss 0.086 | Accuracy 0.823
Epoch 50 | Loss 0.062 | Accuracy 0.808
Epoch 60 | Loss 0.049 | Accuracy 0.808
Epoch 70 | Loss 0.040 | Accuracy 0.808
Epoch 80 | Loss 0.034 | Accuracy 0.808


Epoch 90 | Loss 0.029 | Accuracy 0.808


In [161]:
eval_model(gnn, graph, features, labels)

0.8376383763837638

In [162]:
# Compare with the same model trained on empty graph
gnn = init_model(GNN, features, labels)

emptyGraph = dgl.from_scipy(sparse.csr_matrix((len(labels), len(labels))))

train_model(gnn, emptyGraph, features, labels, n_epochs=0)

eval_model(gnn, emptyGraph, features, labels)

0.12546125461254612

In [163]:
# Add a hidden layer with ReLu activation function (e.g., dimension = 20) and retrain the model. 

class GNN_HIDDEN(nn.Module):
    def __init__(self, dim_input, dim_hidden, dim_output):
        super(GNN_HIDDEN, self).__init__()
        self.conv1 = SAGEConv(dim_input, dim_hidden, 'mean')
        self.conv2 = SAGEConv(dim_hidden, dim_output, 'mean')
        
    def forward(self, graph, features):
        output = self.conv1(graph, features)
        output = F.relu(output)
        output = self.conv2(graph, output)
        return output

def init_model(model, features, labels):
    '''Init the GNN with appropriate dimensions.'''
    dim_input = features.shape[1]
    dim_hidden = 20
    dim_output = len(labels.unique())
    return model(dim_input, dim_hidden, dim_output)

gnn = init_model(GNN_HIDDEN, features, labels)

train_model(gnn, graph, features, labels)

eval_model(gnn, graph, features, labels)

Epoch 00 | Loss 2.077 | Accuracy 0.376


Epoch 10 | Loss 0.216 | Accuracy 0.823
Epoch 20 | Loss 0.051 | Accuracy 0.834
Epoch 30 | Loss 0.012 | Accuracy 0.838
Epoch 40 | Loss 0.005 | Accuracy 0.830
Epoch 50 | Loss 0.003 | Accuracy 0.830
Epoch 60 | Loss 0.002 | Accuracy 0.830
Epoch 70 | Loss 0.001 | Accuracy 0.830
Epoch 80 | Loss 0.001 | Accuracy 0.830
Epoch 90 | Loss 0.001 | Accuracy 0.830


0.8892988929889298

In [164]:
# Compare with a classifier based on heat diffusion

diffusion = DiffusionClassifier()

print(labels)

labels_numpy = labels.numpy()

labels_pred = diffusion.fit_predict(adjacency, labels_numpy)

accuracy = get_accuracy_score(labels_numpy, labels_pred)
print(accuracy)

tensor([2, 2, 1,  ..., 6, 6, 6])
1.0


## Build your own GNN

You will now build your own GNN. We start with the graph convolution layer.

In [165]:
class GraphConvLayer(nn.Module):
    def __init__(self, dim_input, dim_output):
        super(GraphConvLayer, self).__init__()
        self.layer = nn.Linear(dim_input, dim_output)
        
    def forward(self, graph, signal):
        with graph.local_scope():
            # message passing
            graph.ndata['node'] = signal
            graph.update_all(fn.copy_u('node', 'message'), fn.mean('message', 'average'))
            h = graph.ndata['average']
            return self.layer(h)

## To do

* Build a GNN with two layers (hidden layer + output) based on this graph convolution layer.
* Train this GNN and compare the results with the previous one.
* Add the input signal to the output of the graph convolution layer and observe the results.
* Retrain the same GNN without message passing in the first layer.

In [166]:
# Build a GNN with two layers (hidden layer + output) based on this graph convolution layer.

class GNN_HIDDEN(nn.Module):
    def __init__(self, dim_input, dim_hidden, dim_output):
        super(GNN_HIDDEN, self).__init__()
        self.conv1 = GraphConvLayer(dim_input, dim_hidden)
        self.conv2 = GraphConvLayer(dim_hidden, dim_output)
        
    def forward(self, graph, features):
        output = self.conv1(graph, features)
        output = F.relu(output)
        output = self.conv2(graph, output)
        return output
    
def init_model(model, features, labels):
    '''Init the GNN with appropriate dimensions.'''
    dim_input = features.shape[1]
    dim_hidden = 20
    dim_output = len(labels.unique())
    return model(dim_input, dim_hidden, dim_output)

gnn = init_model(GNN_HIDDEN, features, labels)

In [167]:
# Train this GNN and compare the results with the previous one.

train_model(gnn, graph, features, labels)

eval_model(gnn, graph, features, labels)


Epoch 00 | Loss 1.935 | Accuracy 0.280


Epoch 10 | Loss 1.022 | Accuracy 0.753
Epoch 20 | Loss 0.431 | Accuracy 0.841
Epoch 30 | Loss 0.253 | Accuracy 0.841
Epoch 40 | Loss 0.184 | Accuracy 0.823
Epoch 50 | Loss 0.144 | Accuracy 0.819
Epoch 60 | Loss 0.117 | Accuracy 0.823
Epoch 70 | Loss 0.099 | Accuracy 0.827
Epoch 80 | Loss 0.085 | Accuracy 0.830
Epoch 90 | Loss 0.073 | Accuracy 0.830


0.8376383763837638

In [168]:
# Add the input signal to the output of the graph convolution layer and observe the results.

class GNN_HIDDEN(nn.Module):
    def __init__(self, dim_input, dim_hidden, dim_output):
        super(GNN_HIDDEN, self).__init__()
        self.conv1 = GraphConvLayer(dim_input, dim_hidden)
        self.conv2 = GraphConvLayer(dim_hidden, dim_output)
        
    def forward(self, graph, features):
        output = self.conv1(graph, features)
        output = F.relu(output)
        output = self.conv2(graph, output)
        output = output + features
        return output
    
def init_model(model, features, labels):
    '''Init the GNN with appropriate dimensions.'''
    dim_input = features.shape[1]
    dim_hidden = 20
    dim_output = len(labels.unique())
    return model(dim_input, dim_hidden, dim_output)

gnn = init_model(GNN_HIDDEN, features, labels)

train_model(gnn, graph, features, labels)

eval_model(gnn, graph, features, labels)

RuntimeError: The size of tensor a (7) must match the size of tensor b (1433) at non-singleton dimension 1

## Heat diffusion as a GNN

Node classification by heat diffusion can be seen as a GNN without training, using a one-hot encoding of labels. Features are ignored.

## To do

* Build a special GNN whose output corresponds to one step of heat diffusion in the graph.
* Use this GNN to classify nodes by heat diffusion, with temperature centering.

In [169]:
from sknetwork.utils import get_membership

In [170]:
labels_one_hot = get_membership(labels).toarray()
labels_one_hot = torch.Tensor(labels_one_hot)

In [171]:
class Diffusion(nn.Module):
    def __init__(self):
        super(Diffusion, self).__init__()
        
    def forward(self, graph, signal, mask):
        '''Mask is a boolean tensor giving the training set.'''
        with graph.local_scope():
            # message passing
            graph.ndata['node'] = signal
            graph.update_all(fn.copy_u('node', 'message'), fn.mean('message', 'average'))
            signal = graph.ndata['average']
            h = signal
            return h

In [172]:
diffusion = Diffusion()

In [173]:
n_iter = 20

temperatures = labels_one_hot
temperatures[~train] = 0
for t in range(n_iter):
    temperatures = diffusion(graph, temperatures, train)
    
# temperature centering
temperatures -= temperatures.mean(axis=0)

# classification
logits = torch.matmul(temperatures, gnn(graph, features))
labels_pred = torch.argmax(logits, axis=1)

accuracy = get_accuracy_score(labels[train].numpy(), labels_pred[train].numpy())
print(accuracy)

RuntimeError: The size of tensor a (7) must match the size of tensor b (1433) at non-singleton dimension 1

# Quiz

In [19]:
# Import house graph
from sknetwork.data import house
from IPython.display import SVG

house = house(metadata=True)

adjacency = house.adjacency
position = house.position

features = np.array([[0, 1],
            [0, 1],
            [2, 3],
            [-2, -1],
            [2, -1]])


# Each layer consists of the sum of the embedding of the node and the average embedding of the neighbors, followed by a ReLu activation function.
# Consider a neuron of the first layer, with weights w=(1/2,−1) and bias b=1
# Compute the output of this neuron for each node of the graph, from 1 to 5

# Define the weights and bias of the neuron
w = np.array([1/2, -1])
b = 1

adjacency = adjacency.toarray()

P = np.diag(1/adjacency.sum(axis=0)).dot(adjacency)

P = np.eye(5) + P # add self embedding

# Compute the output of the neuron for each node

P = torch.Tensor(P)
features = torch.Tensor(features)
w = torch.Tensor(w)

output = torch.matmul(features, w) + b
print(output)
output = torch.matmul(P, output)
output = torch.relu(output)
output

tensor([ 0.,  0., -1.,  1.,  3.])


tensor([1.5000, 0.6667, 0.0000, 2.0000, 3.3333])